In [1]:
import psycopg2
import pandas as pd

In [2]:
conn = psycopg2.connect(
    host="localhost",
    database="classes",
    user="postgres",
    password="28111230")

In [23]:
SQL_Query = pd.read_sql_query('SELECT * FROM enrollment_data WHERE section_id = 2486 ORDER BY created_at', conn)
SQL_Query.head()

In [48]:
ql_query = pd.read_sql_query(f'SELECT * FROM enrollment_data WHERE section_id = 1823 ORDER BY created_at', conn)
sql_query['diff'] = sql_query['enrollment_count'].diff()
df_filtered = sql_query[sql_query['diff'] != 0]
df_filtered.head()

,id,section_id,enrollment_status,enrollment_count,enrollment_capacity,waitlist_status,waitlist_count,waitlist_capacity,created_at,diff
0,2881026,170,Open,0,40,None,0,0,2019-11-02 15:15:23.909733-07:00,NaN
55,3165336,170,Open,1,40,None,0,0,2019-11-04 21:01:16.940933-08:00,1.0
70,3240128,170,Open,2,40,None,0,0,2019-11-05 12:01:47.715816-08:00,1.0
92,3353117,170,Open,3,40,None,0,0,2019-11-06 10:01:50.855932-08:00,1.0
113,3459309,170,Open,4,40,None,0,0,2019-11-07 07:01:51.303060-08:00,1.0


In [4]:
import requests
import urllib.parse as urlparse
from urllib.parse import urlencode
from bs4 import BeautifulSoup

In [5]:
def _generate_url(base_url, params):
    """
    Generate a URL given many parameters to attach as query strings. Also replaces single quote with double quote
    because that's what UCLA likes.
    """
    url_parts = list(urlparse.urlparse(base_url))
    query = dict(urlparse.parse_qsl(url_parts[4]))
    query.update(params)

    url_parts[4] = urlencode(query)

    final_url = urlparse.urlunparse(url_parts).replace("%27", "%22")
    return final_url

In [22]:
df = pd.read_sql_query("select * from sections where 'Mathematical Sciences 4000A'=ANY(locations);", conn)
df.head()

,id,section_id,course_id,days,times,locations,units,instructors,created_at,updated_at,term,enrollment_status,enrollment_count,enrollment_capacity,waitlist_status,waitlist_count,waitlist_capacity
0,5477091,145708200,522619,[TR],[12:30pm-1:45pm],[Mathematical Sciences 4000A],4.0,"[Simpson, L., Johnson, P.J.]",2020-01-20 12:29:58.371395-08:00,2020-02-25 11:27:12.933339-08:00,10F,Open,106,150,Open,0,12
1,5202195,123009200,3894,[MWF],[11am-11:50am],[Mathematical Sciences 4000A],5.0,"[Furlanetto, S.R.]",2020-01-20 10:52:17.898171-08:00,2020-01-20 11:48:21.626145-08:00,08S,Open,116,140,Open,0,14
2,5881393,262186210,5383,[MWF],[11am-11:50am],[Mathematical Sciences 4000A],4.0,"[Greene, M.P.]",2020-01-20 16:34:35.120034-08:00,2020-03-09 12:07:34.994738-07:00,20S,Open,193,210,Open,5,18
3,5167438,126004200,5596,[TR],[9:30am-10:45am],[Mathematical Sciences 4000A],4.0,"[Baschek, B.G.]",2020-01-20 10:39:13.040918-08:00,2020-01-20 11:33:08.235562-08:00,07F,Open,69,81,Open,0,9
4,5881365,262186200,5383,[MWF],[9am-9:50am],[Mathematical Sciences 4000A],4.0,"[Hoff, D.J.]",2020-01-20 16:34:34.937503-08:00,2020-03-09 12:07:32.598655-07:00,20S,Open,132,140,Open,0,12


In [25]:
PUBLIC_RESULTS_URL     = "https://sa.ucla.edu/ro/Public/SOC/Results"
HEADERS = {"X-Requested-With": "XMLHttpRequest"}
def get_subject(row):
    if row is None:
        return ""
    
    params = {'t': row["term"], 'sBy': 'classidnumber', 'id': row["section_id"]}
    final_url = _generate_url(PUBLIC_RESULTS_URL, params)
    soup = BeautifulSoup(requests.get(final_url, headers=HEADERS).content, "lxml")
    if soup.select_one('a[title^="Class Detail for "]') is None:
        return ""
    details_url = "https://sa.ucla.edu" + soup.select_one('a[title^="Class Detail for "]')['href']
    details_url_parts = dict(urlparse.parse_qsl(list(urlparse.urlparse(details_url))[4]))
    subject = details_url_parts['subj_area_cd'].strip()
    return subject


In [26]:
df['subject'] = df.apply(lambda x: get_subject(x), axis=1)
df.head()

,id,section_id,course_id,days,times,locations,units,instructors,created_at,updated_at,term,enrollment_status,enrollment_count,enrollment_capacity,waitlist_status,waitlist_count,waitlist_capacity,subject
0,5477091,145708200,522619,[TR],[12:30pm-1:45pm],[Mathematical Sciences 4000A],4.0,"[Simpson, L., Johnson, P.J.]",2020-01-20 12:29:58.371395-08:00,2020-02-25 11:27:12.933339-08:00,10F,Open,106,150,Open,0,12,MIMG
1,5202195,123009200,3894,[MWF],[11am-11:50am],[Mathematical Sciences 4000A],5.0,"[Furlanetto, S.R.]",2020-01-20 10:52:17.898171-08:00,2020-01-20 11:48:21.626145-08:00,08S,Open,116,140,Open,0,14,ASTR
2,5881393,262186210,5383,[MWF],[11am-11:50am],[Mathematical Sciences 4000A],4.0,"[Greene, M.P.]",2020-01-20 16:34:35.120034-08:00,2020-03-09 12:07:34.994738-07:00,20S,Open,193,210,Open,5,18,MATH
3,5167438,126004200,5596,[TR],[9:30am-10:45am],[Mathematical Sciences 4000A],4.0,"[Baschek, B.G.]",2020-01-20 10:39:13.040918-08:00,2020-01-20 11:33:08.235562-08:00,07F,Open,69,81,Open,0,9,A&O SCI
4,5881365,262186200,5383,[MWF],[9am-9:50am],[Mathematical Sciences 4000A],4.0,"[Hoff, D.J.]",2020-01-20 16:34:34.937503-08:00,2020-03-09 12:07:32.598655-07:00,20S,Open,132,140,Open,0,12,MATH


In [30]:
df["subject"].value_counts().head(10).to_csv("asf.csv")

In [32]:
df = pd.read_sql_query("select id, section_id, days, times, locations, enrollment_count from sections where 'Mathematical Sciences 4000A'=ANY(locations) and term='20W';", conn)
df.head()

,id,section_id,days,times,locations,enrollment_count
0,2135,262181200,[MWF],[8am-8:50am],[Mathematical Sciences 4000A],74
1,138,262211230,[MWF],[4pm-4:50pm],[Mathematical Sciences 4000A],188
2,874,262074200,[MWF],[9am-9:50am],[Mathematical Sciences 4000A],40
3,3130,262206230,[MWF],[1pm-1:50pm],[Mathematical Sciences 4000A],64
4,3129,262206220,[MWF],[12pm-12:50pm],[Mathematical Sciences 4000A],197


In [33]:
df.to_csv("asf.csv")